In [ ]:
!git clone https://github.com/ultralytics/yolov3
%cd yolov3
%pip install -qr requirements.txt

%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5

%pip install -qr requirements.txt

In [ ]:
## file loading

from google.colab import drive
import shutil
import zipfile

drive.mount('/content/drive')

shutil.copy('/content/drive/My Drive/data/leafdry_with_annotation.zip', '/content/yolov5')

zip_file_train_img='/content/yolov5/leafdry_with_annotation.zip'

with zipfile.ZipFile(zip_file_train_img, 'r') as zip_ref:
  zip_ref.extractall('/content/yolov5/leafdry')

In [ ]:
from IPython.display import Image, display, clear_output
from IPython.core.magic import register_line_cell_magic
import yaml
import torch

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!touch /content/yolov5/leafdry/train/train.txt
!touch /content/yolov5/leafdry/test/test.txt
!touch /content/yolov5/leafdry/validation/validation.txt
!touch /content/yolov5/leafdry/model_validation/model_validation.txt

In [ ]:
df_train=pd.read_csv('/content/yolov5/leafdry/csv_files/annotation_train.csv', encoding='utf-8')
df_test = pd.read_csv('/content/yolov5/leafdry/csv_files/annotation_test.csv', encoding='utf-8')
df_val = pd.read_csv('/content/yolov5/leafdry/csv_files/validation.csv', encoding='utf-8')
df_model_val = pd.read_csv('/content/yolov5/leafdry/csv_files/model_validation.csv', encoding='utf-8')

train_img_list = df_train['img'].to_list()
test_img_list = df_test['img'].to_list()
val_img_list = df_val['img'].to_list()

In [ ]:
## 학습, 테스트 이미지 파일 리스트 txt 파일 생성
import yaml

with open('/content/yolov5/leafdry/train/train.txt', 'w') as f:
  f.write('\n' .join(train_img_list)+'\n')

with open('/content/yolov5/train_dry/test/test.txt', 'w') as f:
  f.write('\n' .join(test_img_list)+'\n')

with open('/content/yolov5/train_dry/validation/valid.txt', 'w') as f:
  f.write('\n' .join(val_img_list)+'\n')

In [ ]:
## 객체탐지를 위한 모듈 임포트 및 정의함수 생성

from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    f.write(cell.format(**globals()))

In [ ]:
!touch /content/yolov5/leafdry/data.yaml

In [ ]:
yaml_content = """
train: /content/yolov5/leafdry/train/train_img
test: /content/yolov5/leafdry/test
val: /content/yolov5/leafdry/validation

nc: 4
names: ['leaf-N', 'stem-N', 'flower-N', 'fruit-N', 'leaf-E', 'stem-E', 'flower-E', 'fruit-E', 'leaf-Y', 'stem-Y', 'flower-Y', 'fruit-Y']
"""

with open('/content/yolov5/leafdry/data.yaml', 'w') as file:
    file.write(yaml_content)

In [ ]:
%cat /content/yolov5/leafdry/data.yaml

In [ ]:
import yaml

with open('/content/yolov5/leafdry/data.yaml', 'r') as steam:
  num_classes = str(yaml.safe_load(steam)['nc'])

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
yaml_content = """
# Parameters
nc: 12  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]
"""

with open('/content/yolov5/models/custom_yolov5s.yaml', 'w') as file:
    file.write(yaml_content)

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml

In [ ]:
%time

!python train.py --img 640 --batch 32 --epochs 50 --data ./leafdry/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name dry_train_result --cache

In [ ]:
%time

!python train.py --img 640 --batch 32 --epochs 100 --data ./leafdry/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name dry_train_result --cache

In [ ]:
Image(filename='/content/yolov5/runs/train/leafdry_result/train_batch0.jpg', width=1200)

In [ ]:
!python val.py --weights runs/train/leafdry_result2/weights/best.pt --data ./leafdry/data.yaml --img 640 --iou 0.65 --half

In [ ]:
!python val.py --weights runs/train/leafdry_result3/weights/best.pt --data ./leafdry/data.yaml --img 640 --iou 0.65 --half

In [ ]:
!python detect.py --weights runs/train/leafdry_result2/weights/best.pt --img 640 --conf 0.2 --source ./leafdry/test

In [ ]:
!python detect.py --weights runs/train/leafdry_result3/weights/best.pt --img 640 --conf 0.2 --source ./leafdry/test

In [ ]:
import random
from IPython.display import Image, display
import glob

Image_name = random.choice(glob.glob('/content/yolov5/runs/val/exp4/*.jpg'))
display(Image(filename=Image_name))

In [ ]:
import random
from IPython.display import Image, display
import glob

Image_name = random.choice(glob.glob('/content/yolov5/runs/detect/exp3/*.jpg'))
display(Image(filename=Image_name))